In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import glob

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# for notebook
%matplotlib inline

## Datasets

copa_do_brasil : 4 temporadas

In [2]:
# https://basedosdados.org/dataset/c861330e-bca2-474d-9073-bc70744a1b23?table=19e96c3f-c932-4e14-82e2-58e170454e3d 

copa_brasil = pd.read_csv('../data/copa_brasil/copa_brasil.csv', sep=',')
df = copa_brasil.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ano_campeonato                  424 non-null    int64  
 1   data                            424 non-null    object 
 2   horario                         424 non-null    object 
 3   fase                            424 non-null    object 
 4   tipo_fase                       424 non-null    object 
 5   estadio                         423 non-null    object 
 6   arbitro                         110 non-null    object 
 7   publico                         424 non-null    int64  
 8   publico_max                     396 non-null    float64
 9   time_mandante                   424 non-null    object 
 10  time_visitante                  424 non-null    object 
 11  tecnico_mandante                262 non-null    object 
 12  tecnico_visitante               262 

In [3]:
df.head(20)

,ano_campeonato,data,horario,fase,tipo_fase,estadio,arbitro,publico,publico_max,time_mandante,...,chutes_bola_parada_mandante,chutes_bola_parada_visitante,defesas_mandante,defesas_visitante,impedimentos_mandante,impedimentos_visitante,chutes_mandante,chutes_visitante,chutes_fora_mandante,chutes_fora_visitante
0,2020,2020-12-23,20:30:00,Semi-Finais,Jogos De Ida,Arena do Grêmio,NaN,0,60540.0,Grêmio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,2020-10-28,18:00:00,Oitavas De Final,Jogos De Ida,Estádio Antonio Accioly,NaN,0,13000.0,Atlético-GO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,2020-10-28,20:30:00,Oitavas De Final,Jogos De Ida,Arena do Grêmio,NaN,0,60540.0,Grêmio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,2020-09-23,21:30:00,4ª Eliminatória,Jogos De Volta,Estádio São Januário,NaN,0,24584.0,Vasco da Gama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,2020-08-25,16:00:00,3ª Eliminatória,Jogos De Volta,Estádio Vianão,NaN,0,1735.0,Afogados,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020,2020-08-25,19:00:00,3ª Eliminatória,Jogos De Volta,Estádio Raimundo Sampaio,NaN,0,25000.0,América-MG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020,2020-08-25,21:30:00,3ª Eliminatória,Jogos De Volta,Estádio Jornalista Mário Filho,NaN,0,82238.0,Fluminense,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020,2020-08-26,16:00:00,3ª Eliminatória,Jogos De Volta,Estádio Rei Pelé,NaN,0,20000.0,CRB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020,2020-08-26,19:00:00,3ª Eliminatória,Jogos De Volta,Estádio Durival Britto e Silva,NaN,0,20083.0,Paraná,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020,2020-08-26,21:30:00,3ª Eliminatória,Jogos De Volta,Estádio Manoel Barradas,NaN,0,35632.0,EC Vitória,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.tail(20)

,ano_campeonato,data,horario,fase,tipo_fase,estadio,arbitro,publico,publico_max,time_mandante,...,chutes_bola_parada_mandante,chutes_bola_parada_visitante,defesas_mandante,defesas_visitante,impedimentos_mandante,impedimentos_visitante,chutes_mandante,chutes_visitante,chutes_fora_mandante,chutes_fora_visitante
404,2022,2022-06-22,19:30:00,Oitavas De Final,Jogos De Ida,Arena Fonte Nova,Braulio da Silva Machado,20484,55000.0,EC Bahia,...,10.0,19.0,3.0,3.0,0.0,2.0,18.0,10.0,7.0,2.0
405,2022,2022-10-19,21:45:00,Final,Jogo De Volta,Estádio Jornalista Mário Filho,NaN,61566,78838.0,Flamengo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
406,2022,2022-07-14,20:00:00,Oitavas De Final,Jogos De Volta,Allianz Parque,NaN,41361,43600.0,Palmeiras,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407,2022,2022-05-10,20:30:00,3ª Eliminatória,Jogos De Volta,Os Pioneiros,NaN,6029,8000.0,Azuriz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,2022,2022-05-11,22:00:00,3ª Eliminatória,Jogos De Volta,Arena Pantanal,NaN,9156,42968.0,Cuiabá-MT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
409,2022,2022-05-12,19:30:00,3ª Eliminatória,Jogos De Volta,Estádio Raimundo Sampaio,NaN,22158,23000.0,Cruzeiro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,2022,2022-05-31,21:30:00,3ª Eliminatória,Jogos De Volta,Estádio Nabi Abi Chedid,NaN,1668,13212.0,RB Bragantino,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,2022,2022-03-08,20:30:00,2ª Eliminatória,Jogo Único,Estádio Orlando Scarpelli,NaN,6066,19908.0,Figueirense FC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,2022,2022-03-08,20:30:00,2ª Eliminatória,Jogo Único,Estádio Governador Alberto Tavares Silva,NaN,2991,44200.0,Fluminense-PI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413,2022,2022-03-09,20:30:00,2ª Eliminatória,Jogo Único,Estádio Adauto Moraes,NaN,2,5018.0,Juazeirense,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['ano_campeonato', 'data', 'horario', 'fase', 'tipo_fase', 'estadio',
       'arbitro', 'publico', 'publico_max', 'time_mandante', 'time_visitante',
       'tecnico_mandante', 'tecnico_visitante',
       'valor_equipe_titular_mandante', 'valor_equipe_titular_visitante',
       'idade_media_titular_mandante', 'idade_media_titular_visitante',
       'gols_mandante', 'gols_visitante', 'gols_1_tempo_mandante',
       'gols_1_tempo_visitante', 'penalti', 'gols_penalti_mandante',
       'gols_penalti_visitante', 'escanteios_mandante', 'escanteios_visitante',
       'faltas_mandante', 'faltas_visitante', 'chutes_bola_parada_mandante',
       'chutes_bola_parada_visitante', 'defesas_mandante', 'defesas_visitante',
       'impedimentos_mandante', 'impedimentos_visitante', 'chutes_mandante',
       'chutes_visitante', 'chutes_fora_mandante', 'chutes_fora_visitante'],
      dtype='object')

In [6]:
df = df[[
    'ano_campeonato', 
    'time_mandante', 
    'time_visitante', 
    'gols_mandante', 
    'gols_visitante'
]]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ano_campeonato  424 non-null    int64 
 1   time_mandante   424 non-null    object
 2   time_visitante  424 non-null    object
 3   gols_mandante   424 non-null    int64 
 4   gols_visitante  424 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 16.7+ KB


In [7]:
df.rename(columns = {
    'ano_campeonato':'year', 
    'time_mandante':'home', 
    'time_visitante':'away', 
    'gols_mandante':'home_goals', 
    'gols_visitante':'away_goals'
    }, inplace = True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        424 non-null    int64 
 1   home        424 non-null    object
 2   away        424 non-null    object
 3   home_goals  424 non-null    int64 
 4   away_goals  424 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 16.7+ KB


In [8]:
df = df[['year','home','away','home_goals','away_goals']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        424 non-null    int64 
 1   home        424 non-null    object
 2   away        424 non-null    object
 3   home_goals  424 non-null    int64 
 4   away_goals  424 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 16.7+ KB


In [9]:
df = df.dropna(how='any',axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        424 non-null    int64 
 1   home        424 non-null    object
 2   away        424 non-null    object
 3   home_goals  424 non-null    int64 
 4   away_goals  424 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 16.7+ KB


In [10]:
print(sorted(df['year'].unique()))
print("#temporadas = ", len(df['year'].unique()))

[2020, 2021, 2022, 2023]
#temporadas =  4


In [11]:
#temp = ['2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23']
#df = df.loc[df['year'].isin(temp)].reset_index()

In [12]:
vhome = list(df['home'].unique())
print(len(vhome))

vaway = list(df['away'].unique())
print(len(vaway))

vertice = set(vhome) | set(vaway)
print(len(vertice))
print(list(vertice))

161
90
170
['Portuguesa-RJ', 'Costa Rica', 'Operário-PR', 'Guarani', 'Vasco da Gama', 'Oeste', 'Barbalha', 'Moto Club', 'Princesa-AM', 'Palmas', 'Marília', 'América-MG', 'Peñarol-AM', 'Athletico-PR', 'Vilhenense', 'Imperatriz', 'XV Piracicaba', 'Ponte Preta', 'Boavista', 'Falcon', 'Aquidauanense', 'Cruzeiro', 'Pouso Alegre', 'União Rond.', 'SE Gama', 'Ferroviária', 'Náutico', 'Atlético-MG', 'Picos', 'Londrina-PR', 'Ceilândia', 'Retrô', 'FC Cascavel', 'Nova Iguaçu', 'Cianorte', 'RB Bragantino', 'Santo André', 'Bangu-RJ', 'América-RN', 'Freipaulistano', 'AD Bahia-BA', 'Bragantino-PA', 'Ituano', 'Ceará SC', 'Grêmio', 'Uberlândia', 'U. Rondonópolis', 'Globo', 'São Luiz', 'São Bernardo', 'Iguatu', 'Parnahyba', 'CRB', 'Operário-MS', 'EC Vitória', 'Castanhal', 'Rio Branco VN', 'Rio Branco-AC', 'Galvez', 'Santa Cruz-RS', 'Real Brasília', 'Anápolis', 'River-PI', 'Athletic', 'Fluminense', 'CSA', 'Figueirense FC', 'Santos', 'Humaitá', 'Coruripe', 'Atlético-GO', 'Luverdense', 'Goianésia (GO)', 'No

In [13]:
df['home_win'] = 0.0
df['away_win'] = 0.0
df['draw'] = 0.0

df.loc[df['home_goals'] > df['away_goals'], 'home_win'] = 1.0
df.loc[df['home_goals'] < df['away_goals'], 'away_win'] = 1.0
df.loc[df['home_goals'] == df['away_goals'], 'draw'] = 1.0

df.head()

,year,home,away,home_goals,away_goals,home_win,away_win,draw
0,2020,Grêmio,São Paulo,1,0,1.0,0.0,0.0
1,2020,Atlético-GO,Internacional,1,2,0.0,1.0,0.0
2,2020,Grêmio,Juventude,1,0,1.0,0.0,0.0
3,2020,Vasco da Gama,Botafogo,0,0,0.0,0.0,1.0
4,2020,Afogados,Ponte Preta,0,2,0.0,1.0,0.0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        424 non-null    int64  
 1   home        424 non-null    object 
 2   away        424 non-null    object 
 3   home_goals  424 non-null    int64  
 4   away_goals  424 non-null    int64  
 5   home_win    424 non-null    float64
 6   away_win    424 non-null    float64
 7   draw        424 non-null    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 26.6+ KB


In [15]:
df2 = df.groupby(
    ['home', 'away'],
    as_index=False,
    sort=False).agg({'home_goals':'sum', 'away_goals':'sum', 'home_win':'sum', 'away_win':'sum', 'draw' : 'sum'})

df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
0,Grêmio,São Paulo,1,0,1.0,0.0,0.0
1,Atlético-GO,Internacional,1,2,0.0,1.0,0.0
2,Grêmio,Juventude,1,0,1.0,0.0,0.0
3,Vasco da Gama,Botafogo,0,0,0.0,0.0,1.0
4,Afogados,Ponte Preta,0,2,0.0,1.0,0.0
5,América-MG,Ferroviária,1,0,1.0,0.0,0.0
6,Fluminense,Figueirense FC,3,0,1.0,0.0,0.0
7,CRB,Cruzeiro,1,1,0.0,0.0,1.0
8,Paraná,Botafogo,1,2,0.0,1.0,0.0
9,EC Vitória,Ceará SC,3,4,0.0,1.0,0.0


In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409 entries, 0 to 408
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   home        409 non-null    object 
 1   away        409 non-null    object 
 2   home_goals  409 non-null    int64  
 3   away_goals  409 non-null    int64  
 4   home_win    409 non-null    float64
 5   away_win    409 non-null    float64
 6   draw        409 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 22.5+ KB


In [18]:
lista = []
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away
    t = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)][['home','away']]
    
    if len(t) == 0:
        lista.append(time1)
        print(time2,time1)

print(len(lista))

Figueirense FC Vitória-ES
Remo Brusque
AD Bahia-BA Paraná
Águia Negra Ferroviária
Botafogo-PB Fluminense
Goiás Santo André
Manaus FC Brasil Pelotas
Lagarto EC Vitória
América-MG Operário-PR
ABC FC Vasco da Gama
Juventude Coruripe
Criciúma EC Santo André
Santa Cruz Operário VG
Londrina-PR XV Piracicaba
Boa Esporte Vilhenense
Botafogo-PB Atl. Alagoinhas
Operário-PR Barbalha
América-RN São Luiz
Luverdense AD Bahia-BA
CRB Independente-PA
América-MG Santos-AP
EC Bahia River-PI
Paraná Palmas
ABC FC Aquidauanense
Botafogo Caxias-RS
Goiás Fast
Figueirense FC Novorizontino
Paysandu Brasiliense
CSA Vitória-ES
Atlético-GO União Rond.
EC Vitória Imperatriz
Ceará SC Bragantino-PA
Sport Recife Brusque
Vasco da Gama Altos
Oeste Bangu-RJ
Remo Freipaulistano
Brasil Pelotas SE Gama
EC São José Caucaia-CE
Náutico Toledo
Atlético-MG Campinense-PB
Sampaio Corrêa Águia Negra
Fluminense Moto Club
Vila Nova FC Galvez
Coritiba FC Manaus FC
Volta Redonda Lagarto
Avaí FC Ferroviária
Ponte Preta Novo Hamburgo
Atl

In [20]:
time1 = "Figueirense FC"
time2 = "Vitória-ES"

t = df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw']]
print(t.values)

t = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw']]
print(t.values)

[]
[['Vitória-ES' 'Figueirense FC' 0 1 0.0 1.0 0.0]]


In [21]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
0,Grêmio,São Paulo,1,0,1.0,0.0,0.0
1,Atlético-GO,Internacional,1,2,0.0,1.0,0.0
2,Grêmio,Juventude,1,0,1.0,0.0,0.0
3,Vasco da Gama,Botafogo,0,0,0.0,0.0,1.0
4,Afogados,Ponte Preta,0,2,0.0,1.0,0.0
5,América-MG,Ferroviária,1,0,1.0,0.0,0.0
6,Fluminense,Figueirense FC,3,0,1.0,0.0,0.0
7,CRB,Cruzeiro,1,1,0.0,0.0,1.0
8,Paraná,Botafogo,1,2,0.0,1.0,0.0
9,EC Vitória,Ceará SC,3,4,0.0,1.0,0.0


In [22]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
389,América-MG,São Paulo,2,2,0.0,0.0,1.0
390,EC Bahia,Athletico-PR,1,2,0.0,1.0,0.0
391,Flamengo,Corinthians,1,1,0.0,0.0,1.0
392,Palmeiras,São Paulo,2,1,1.0,0.0,0.0
393,Azuriz,EC Bahia,1,1,0.0,0.0,1.0
394,Cuiabá-MT,Atlético-GO,0,0,0.0,0.0,1.0
395,Cruzeiro,Remo,1,0,1.0,0.0,0.0
396,RB Bragantino,Goiás,0,1,0.0,1.0,0.0
397,Figueirense FC,Cuiabá-MT,2,2,0.0,0.0,1.0
398,Fluminense-PI,Santos,1,1,0.0,0.0,1.0


In [23]:
df2["weight1"] = 0.0
df2["weight2"] = 0.0

In [24]:
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away

    t1 =  df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['away_goals','away_win','draw']].values
    t2 =  df2.loc[(df2['away'] == time1) & (df2['home'] == time2)][['home_goals','home_win','draw']].values

    g1 = p1 = 0
    for i1 in t1:
        g1 = i1[0]
        p1 = 3*i1[1] + 1*i1[2]

    g2 = p2 = 0
    for i2 in t2:
        g2 = i2[0]
        p2 = 3*i2[1] + 1*i2[2]

    df2.loc[keys,'weight1'] = g1 + g2
    df2.loc[keys,'weight2'] = p1 + p2

In [25]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
0,Grêmio,São Paulo,1,0,1.0,0.0,0.0,0.0,1.0
1,Atlético-GO,Internacional,1,2,0.0,1.0,0.0,4.0,6.0
2,Grêmio,Juventude,1,0,1.0,0.0,0.0,0.0,0.0
3,Vasco da Gama,Botafogo,0,0,0.0,0.0,1.0,1.0,4.0
4,Afogados,Ponte Preta,0,2,0.0,1.0,0.0,5.0,6.0
5,América-MG,Ferroviária,1,0,1.0,0.0,0.0,0.0,1.0
6,Fluminense,Figueirense FC,3,0,1.0,0.0,0.0,1.0,3.0
7,CRB,Cruzeiro,1,1,0.0,0.0,1.0,1.0,1.0
8,Paraná,Botafogo,1,2,0.0,1.0,0.0,3.0,6.0
9,EC Vitória,Ceará SC,3,4,0.0,1.0,0.0,5.0,6.0


In [26]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
389,América-MG,São Paulo,2,2,0.0,0.0,1.0,3.0,4.0
390,EC Bahia,Athletico-PR,1,2,0.0,1.0,0.0,4.0,6.0
391,Flamengo,Corinthians,1,1,0.0,0.0,1.0,1.0,2.0
392,Palmeiras,São Paulo,2,1,1.0,0.0,0.0,2.0,3.0
393,Azuriz,EC Bahia,1,1,0.0,0.0,1.0,1.0,2.0
394,Cuiabá-MT,Atlético-GO,0,0,0.0,0.0,1.0,1.0,2.0
395,Cruzeiro,Remo,1,0,1.0,0.0,0.0,2.0,3.0
396,RB Bragantino,Goiás,0,1,0.0,1.0,0.0,2.0,3.0
397,Figueirense FC,Cuiabá-MT,2,2,0.0,0.0,1.0,2.0,1.0
398,Fluminense-PI,Santos,1,1,0.0,0.0,1.0,1.0,1.0


In [27]:
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away

    t2 = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)]
    
    if len(t2) == 0:
        t1 =  df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['home_goals','home_win','draw']].values

        w1 = w2 = 0
        for i1 in t1:
            w1 = i1[0]
            w2 = 3*i1[1] + 1*i1[2]

        new = [time2, time1,np.nan,np.nan,0,0,0,w1,w2]

        new = pd.DataFrame(columns=df2.columns, data=[new])
        
        df2 = pd.concat([df2, new], axis=0)

In [28]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
0,Grêmio,São Paulo,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,Atlético-GO,Internacional,1.0,2.0,0.0,1.0,0.0,4.0,6.0
2,Grêmio,Juventude,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Vasco da Gama,Botafogo,0.0,0.0,0.0,0.0,1.0,1.0,4.0
4,Afogados,Ponte Preta,0.0,2.0,0.0,1.0,0.0,5.0,6.0
5,América-MG,Ferroviária,1.0,0.0,1.0,0.0,0.0,0.0,1.0
6,Fluminense,Figueirense FC,3.0,0.0,1.0,0.0,0.0,1.0,3.0
7,CRB,Cruzeiro,1.0,1.0,0.0,0.0,1.0,1.0,1.0
8,Paraná,Botafogo,1.0,2.0,0.0,1.0,0.0,3.0,6.0
9,EC Vitória,Ceará SC,3.0,4.0,0.0,1.0,0.0,5.0,6.0


In [29]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
0,EC Bahia,Jacuipense-BA,NaN,NaN,0.0,0.0,0.0,1.0,0.0
0,Ceará SC,Caldense,NaN,NaN,0.0,0.0,0.0,0.0,0.0
0,Botafogo-PB,Águia de Marabá,NaN,NaN,0.0,0.0,0.0,2.0,3.0
0,Criciúma EC,Real Ariquemes,NaN,NaN,0.0,0.0,0.0,0.0,0.0
0,América-RN,Iguatu,NaN,NaN,0.0,0.0,0.0,1.0,3.0
0,Sampaio Corrêa,Maringá,NaN,NaN,0.0,0.0,0.0,2.0,3.0
0,Botafogo,Sergipe,NaN,NaN,0.0,0.0,0.0,1.0,1.0
0,RB Bragantino,Ypiranga-RS,NaN,NaN,0.0,0.0,0.0,3.0,3.0
0,Cuiabá-MT,Figueirense FC,NaN,NaN,0.0,0.0,0.0,2.0,1.0
0,Santos,Fluminense-PI,NaN,NaN,0.0,0.0,0.0,1.0,1.0


In [30]:
time1 = "Figueirense FC"
time2 = "Vitória-ES"

t = df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw','weight1','weight2']]
print(t.values)

t = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw','weight1','weight2']]
print(t.values)

[['Figueirense FC' 'Vitória-ES' nan nan 0.0 0.0 0.0 0.0 0.0]]
[['Vitória-ES' 'Figueirense FC' 0.0 1.0 0.0 1.0 0.0 1.0 3.0]]


In [32]:
df3 = df2[['home','away','weight1','weight2']]

df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 646 entries, 0 to 0
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   home     646 non-null    object 
 1   away     646 non-null    object 
 2   weight1  646 non-null    float64
 3   weight2  646 non-null    float64
dtypes: float64(2), object(2)
memory usage: 25.2+ KB


In [33]:
df3.head(20)

,home,away,weight1,weight2
0,Grêmio,São Paulo,0.0,1.0
1,Atlético-GO,Internacional,4.0,6.0
2,Grêmio,Juventude,0.0,0.0
3,Vasco da Gama,Botafogo,1.0,4.0
4,Afogados,Ponte Preta,5.0,6.0
5,América-MG,Ferroviária,0.0,1.0
6,Fluminense,Figueirense FC,1.0,3.0
7,CRB,Cruzeiro,1.0,1.0
8,Paraná,Botafogo,3.0,6.0
9,EC Vitória,Ceará SC,5.0,6.0


In [34]:
df3.tail(20)

,home,away,weight1,weight2
0,EC Bahia,Jacuipense-BA,1.0,0.0
0,Ceará SC,Caldense,0.0,0.0
0,Botafogo-PB,Águia de Marabá,2.0,3.0
0,Criciúma EC,Real Ariquemes,0.0,0.0
0,América-RN,Iguatu,1.0,3.0
0,Sampaio Corrêa,Maringá,2.0,3.0
0,Botafogo,Sergipe,1.0,1.0
0,RB Bragantino,Ypiranga-RS,3.0,3.0
0,Cuiabá-MT,Figueirense FC,2.0,1.0
0,Santos,Fluminense-PI,1.0,1.0


In [36]:
df3.to_csv('../data/work1/copa_do_brasil_2020_2023.csv', index=False, sep=';') 